In [2]:
import numpy as np
import pandas as pd
import sqlite3
import math

In [3]:
connect = sqlite3.connect(r"C:\Users\seans\Desktop\CS 360 - Final Project\data.sqlite3")

In [4]:
# get artists data
artists_query = '''
SELECT *
FROM artists
'''
artists_df = pd.read_sql_query(artists_query, connect)

In [5]:
# get reviews data
reviews_query = '''
SELECT *
FROM reviews
'''
reviews_df = pd.read_sql_query(reviews_query, connect)

In [6]:
# get scores data
scores_query = '''
SELECT *
FROM tombstones
'''
scores_df = pd.read_sql_query(scores_query, connect)

In [7]:
# get genre data
genre_query = '''
SELECT *
FROM genre_review_map
'''
genre_df = pd.read_sql_query(genre_query, connect)

In [8]:
# get artist map data
artist_map_query = '''
SELECT * 
FROM artist_review_map
'''

artist_mapdf = pd.read_sql_query(artist_map_query, connect)

In [9]:
# get genre map data
genre_query = '''
SELECT * 
FROM genre_review_map
'''

genre_df = pd.read_sql_query(genre_query, connect)

In [10]:
# create new data from queries

# need list of columns 
# 'name','artist_id' from artists_df
# 'review_url', 'pub_date' from reviews
# 'title' 'score' from scores_df

# when need to merge df's one by one to make sure the artists, title, and url all match up

In [11]:
# start with artists datafram
merge_df = artists_df.copy()

In [12]:
merge_df

,artist_id,name,artist_url
0,4847-avarus,Avarus,/artists/4847-avarus/
1,33420-pinegrove,Pinegrove,/artists/33420-pinegrove/
2,2438-le-tigre,Le Tigre,/artists/2438-le-tigre/
3,4158-paul-simon,Paul Simon,/artists/4158-paul-simon/
4,4117-alina-simone,Alina Simone,/artists/4117-alina-simone/
...,...,...,...
11008,33868-nottz,Nottz,/artists/33868-nottz/
11009,29248-hilly-eye,Hilly Eye,/artists/29248-hilly-eye/
11010,pom-pom-squad,Pom Pom Squad,/artists/pom-pom-squad/
11011,bruiser-wolf,Bruiser Wolf,/artists/bruiser-wolf/


In [13]:
# merge artist_mapdf column based on artist_id
merge_df = merge_df.merge(artist_mapdf, on='artist_id')

In [14]:
# merge scores_df columns based on review_id
merge_df = merge_df.merge(scores_df, on='review_url')

In [15]:
# merge reviews_df columns based on review_url
merge_df = merge_df.merge(reviews_df, on='review_url')

In [16]:
# merge genre_df columns based on review url
merge_df = merge_df.merge(genre_df, on='review_url')

In [17]:
# drop columns we don't need
merge_df = merge_df.drop(columns=['picker_index','review_tombstone_id','body'])

In [18]:
# reset index
merge_df =  merge_df.reset_index()

In [19]:
# add new column year split from the pub_date column
merge_df['year'] = merge_df['pub_date'].str.split("-").str.get(0)

In [20]:
# too reduce noise and provide better analysis of the data we'll remove music reviews on reissued albums
# shape of merged dataframe before removing data on music reviews for reissues is
# (28588, 13)
merge_df.shape

(28588, 13)

In [21]:
# this removes albums that are considered reissues
merge_df = merge_df[merge_df['is_standard_review'] != 0]

In [23]:
# after removing reissues
# (25266, 13)
merge_df.shape

(25266, 13)

In [24]:
merge_df

,index,artist_id,name,artist_url,review_url,title,score,best_new_music,best_new_reissue,is_standard_review,pub_date,genre,year
0,0,4847-avarus,Avarus,/artists/4847-avarus/,/reviews/albums/9232-vesikansi/,Vesikansi,7.1,0.0,NaN,1,2006-07-27 06:00:04,Experimental,2006
1,1,4847-avarus,Avarus,/artists/4847-avarus/,/reviews/albums/9232-vesikansi/,Vesikansi,7.1,0.0,NaN,1,2006-07-27 06:00:04,Rock,2006
2,2,33420-pinegrove,Pinegrove,/artists/33420-pinegrove/,/reviews/albums/22823-elsewhere/,Elsewhere,7.5,0.0,0.0,1,2017-01-27 06:00:00,Rock,2017
3,3,33420-pinegrove,Pinegrove,/artists/33420-pinegrove/,/reviews/albums/pinegrove-skylight/,Skylight,7.5,0.0,0.0,1,2018-09-29 05:00:00,Rock,2018
4,4,33420-pinegrove,Pinegrove,/artists/33420-pinegrove/,/reviews/albums/21507-cardinal/,Cardinal,8.0,0.0,0.0,1,2016-02-10 06:00:01,Rock,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28583,28583,32955-belly,Belly,/artists/32955-belly/,/reviews/albums/belly-dove/,DOVE,7.2,0.0,0.0,1,2018-05-08 05:00:00,Rock,2018
28584,28584,27785-casper-the-cookies,Casper & the Cookies,/artists/27785-casper-the-cookies/,/reviews/albums/13032-modern-silence/,Modern Silence,5.4,0.0,0.0,1,2009-05-14 06:00:04,Rock,2009
28585,28585,pom-pom-squad,Pom Pom Squad,/artists/pom-pom-squad/,/reviews/albums/various-artists-death-of-a-che...,Death of a Cheerleader,6.9,0.0,0.0,1,2021-06-30 04:00:00,Rock,2021
28586,28586,bruiser-wolf,Bruiser Wolf,/artists/bruiser-wolf/,/reviews/albums/bruiser-wolf-dope-game-stupid/,Dope Game Stupid,7.6,0.0,0.0,1,2021-04-08 04:00:00,Rap,2021


In [25]:
# get the average review score per year
avg_score_per_year = merge_df[['year','score']]

In [26]:
# compute the mean score groupby year
avg_score_per_year = avg_score_per_year.groupby(avg_score_per_year.year).mean()
avg_score_per_year = avg_score_per_year.reset_index()

In [43]:
# export average score per year to csv
avg_score_per_year.to_csv('pfk-avg-score-per-year.csv', header=True, index=False)

In [23]:
# get the average review score per music genre
avg_score_per_genre = merge_df[['genre','score']]

In [24]:
avg_score_per_genre = avg_score_per_genre.groupby(avg_score_per_genre.genre).mean()
avg_score_per_genre = avg_score_per_genre.reset_index()

In [47]:
avg_score_per_genre.to_csv('genre-average-score.csv', header=True, index=False)

In [26]:
# get all the review scores for each music genre over time
genre_trend_over_time = merge_df[['genre','score','pub_date','name','title','year']].copy()

In [27]:
genre_trend_over_time['genre'].value_counts()

Rock            10448
Electronic       4653
Rap              2612
Experimental     2551
Pop/R&B          2081
Folk/Country     1024
Metal             981
Jazz              675
Global            241
Name: genre, dtype: int64

In [28]:
# change music genre string 'Folk/County' to 'Folk_and_County'
# change music genre string 'Pop and R&B' to 'Pop_and_RnB'
# this is due to difficulties in the visualizing special character strings in d3

genre_trend_over_time['genre'] = genre_trend_over_time['genre'].apply(lambda x: 'Pop_and_RnB' if x == 'Pop/R&B' else x)
genre_trend_over_time['genre'] = genre_trend_over_time['genre'].apply(lambda x: 'Folk_and_Country' if x == 'Folk/Country' else x)

In [29]:
genre_trend_over_time['genre'].value_counts()

Rock                10448
Electronic           4653
Rap                  2612
Experimental         2551
Pop_and_RnB          2081
Folk_and_Country     1024
Metal                 981
Jazz                  675
Global                241
Name: genre, dtype: int64

In [30]:
genre_trend_over_time.to_csv("genre-score-over-time.csv", header=True, index=False)

In [61]:
# get the top 5 albums for each genre 
# will be used for tooltip information in the treemap visualisation 
top_rate_by_genre = (merge_df
                     .groupby(["genre", "name",'title','score','year'])
                     .mean()
                     .sort_values(["genre", "score"],
                                                ascending=False)
                     .groupby(level=0, as_index=False)
                     .apply(lambda x: x.head() if len(x) >= 10 else x.head(10))
                     .reset_index(level=0, drop=True)
                    ).round(2)

In [62]:
top_rate_by_genre = top_rate_by_genre.reset_index()

In [64]:
# remove columns we don't need for tooltip
top_rate_by_genre_final = top_rate_by_genre[['genre','name','title','year','score']].copy()

In [65]:
# export top 5 albums by genre 
top_rate_by_genre_final.to_csv("top-five-albums.csv", header=True, index=False)